# 作业3 CNN 食物图片分类
参考:https://blog.csdn.net/iteapoy/article/details/105765411

通过卷积神经网络（Convolutional Neural Networks, CNN）对食物图片进行分类。

数据集中的食物图采集于网上，总共11类：Bread, Dairy product, Dessert, Egg, Fried food, Meat, Noodles/Pasta, Rice, Seafood, Soup, Vegetable/Fruit. 每一类用一个数字表示。
比如：0表示Bread,1,2...10以此类推


In [1]:
#依赖
import numpy as np
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import time
import os

## 数据处理
下载并解压food-11.zip文件,里面有3个文件夹:
    
- training: 9866张
- validation: 3430张
- testing: 3347张

训练集和验证集图片,命名格式为: [类别]_[编号].jpg  
测试集图片命名格式为: [编号].jpg  
输出格式为:
```
Id,Category  
0,0  
1,0  
2,0  
...  
```

### 读取图片
用cv2库读取图片,存储在numpy中

In [2]:
# 读取图片
def readfile(path,label):
    #path  图片路径
    #label 布尔值,表示需不需要返回y值
    
    # 读取目录下所有文件名
    image_dir=sorted(os.listdir(path))
    # x 存储图片 128x128x3 宽高128像素,RGB3色
    x=np.zeros( (len(image_dir),128,128,3), dtype=np.uint8 )
    # y 存储label
    y=np.zeros( (len(image_dir)), dtype=np.uint8 )
    for i,file in enumerate(image_dir):
        img=cv2.imread(os.path.join(path,file))
        # cv2.resize() 将图片大小统一为128x128
        x[i,:,:]=cv2.resize(img,(128,128))
        if label:
            y[i]=int(file.split("_")[0])
    if label:
        return x,y
    else:
        return x

In [3]:
#读取测试集
workspace_dir="./food-11"
print("Reading data ...")
train_x, train_y = readfile(os.path.join(workspace_dir,"training"),True)
print("Size of Training data: {}".format(len(train_x)))

Reading data ...
Size of Training data: 9866


In [4]:
val_x, val_y = readfile(os.path.join(workspace_dir,"validation"),True)
print("Size of Validation data: {}".format(len(val_x)))

Size of Validation data: 3430


In [5]:
test_x = readfile(os.path.join(workspace_dir,"testing"),False)
print("Size of testing: {}".format(len(test_x)))

Size of testing: 3347


### 定义Dataset
Dataset 需要 overload 两个函数：__len__ 及 __getitem__

__len__ 必须要传回 dataset 的大小
__getitem__ 则定义了当函数利用 [ ] 取值时，dataset 应该要怎么传回数据。
实际上，在我们的代码中并不会直接使用到这两个函数，但是当 DataLoader 在 enumerate Dataset 时会使用到，如果没有这样做，程序运行阶段会报错。

这里还对图片进行了数据增强。transforms表示对图片的预处理方式。

In [6]:
#training 时，通过随机旋转、水平翻转图片来进行数据增强（data augmentation）
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(), #随机翻转图片
    transforms.RandomRotation(15), #随机旋转图片
    transforms.ToTensor(), #将图片变成 Tensor，并且把数值normalize到[0,1]
])
#testing 时，不需要进行数据增强（data augmentation）
test_transform = transforms.Compose([
    transforms.ToPILImage(),                                    
    transforms.ToTensor(), #将图片变成 Tensor，并且把数值normalize到[0,1]
])

class ImgDataset(Dataset):
    def __init__(self, x, y=None, transform=None):
        self.x = x
        # label 需要是 LongTensor 型
        self.y = y
        if y is not None:
            #None表示空,y非空时,转换为LongTensor 型
            self.y = torch.LongTensor(y)
        self.transform = transform
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index):
        X = self.x[index]
        if self.transform is not None:
            X = self.transform(X)
        if self.y is not None:
            Y = self.y[index]
            return X, Y
        else:
            return X


In [7]:
batch_size=64
train_set= ImgDataset(train_x,train_y,train_transform)
val_set= ImgDataset(val_x,val_y,test_transform)
# 调用DataLoader,传入训练DataSet,batch size,每个epoch前做一次数据重排
train_loader=DataLoader(train_set,batch_size=batch_size, shuffle=True)
val_loader=DataLoader(val_set,batch_size=batch_size,shuffle=False)

## 定义模型
卷积神经网络CNN+全连接前向传播神经网络FC  

CNN的每一层的结构是:
- 卷积层cov
- 标准池化层 batchnorm
- 激活函数 Relu
- 最大池化层 MaxPllo

In [8]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier,self).__init__()
        # CNN
        # input 3x128x128
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        #torch.nn.MaxPool2d(kernel_size, stride, padding)
        self.cnn = nn.Sequential(
            # 3个input channel,64个3x3 filter,stride为1,padding为1
            nn.Conv2d(3,64,3,1,1),  #输出64x128x128,因为padding+2,3x3-2
            nn.BatchNorm2d(64),     #channel数为64,归一化
            nn.ReLU(),
            nn.MaxPool2d(2,stride=2), #2x2 ,stride=2 输出64x64x64
            
            nn.Conv2d(64,128,3,1,1),  #输出128x64x64
            nn.BatchNorm2d(128),     
            nn.ReLU(),
            nn.MaxPool2d(2,stride=2), #输出128x32x32
            
            nn.Conv2d(128,256,3,1,1),  #输出256x32x32
            nn.BatchNorm2d(256),     
            nn.ReLU(),
            nn.MaxPool2d(2,stride=2), #输出256x16x16
            
            nn.Conv2d(256,512,3,1,1),  #输出512x16x16
            nn.BatchNorm2d(512), 
            nn.ReLU(),
            nn.MaxPool2d(2,stride=2), #输出512x8x8
            
            nn.Conv2d(512,512,3,1,1),  #输出512x8x8
            nn.BatchNorm2d(512), 
            nn.ReLU(),
            nn.MaxPool2d(2,stride=2), #输出512x4x4
        )
        
        #FC
        # input 512x4x4 flatten以后输入
        self.fc=nn.Sequential(
            nn.Linear(512*4*4,1024), #输出1024维
            nn.ReLU(),
            nn.Linear(1024,512),
            nn.ReLU(),
            nn.Linear(512,11),    #输出11维,分类结果
        )
        
    def forward(self,x):
        out=self.cnn(x)
        out=out.view(out.size()[0],-1) #flatten
        return self.fc(out)

## 训练
用training set训练,并用validation set选择最好的参数

In [9]:
model=Classifier().cuda() #GPU加速
loss=nn.CrossEntropyLoss() #分类,使用交叉熵计算损失函数
# CrossEntrophyLoss toolkit里面包含了逻辑回归和softmax,直接就可以分类
optmizer=torch.optim.Adam(model.parameters(),lr=0.001)#优化器Adam 学习速率
num_epoch=30 #迭代次数

for epoch in range(num_epoch):
    epoch_start_time=time.time()
    #初始化准确率和损失为0
    train_acc=0.0
    train_loss=0.0
    val_acc=0.0
    val_loss=0.0
    
    #训练模型
    model.train()
    for i,data in enumerate(train_loader):
        # data[0]是X data[1]是Y
        optmizer.zero_grad() #optim内置函数,把梯度归0
        train_pred=model(data[0].cuda()) #实际上是调用forward函数
        batch_loss=loss(train_pred,data[1].cuda()) #需要参数同在GPU上才能运算
        batch_loss.backward() #计算gradient
        optmizer.step() #optimizer用gradient更新参数
        
        #计算准确率 predict输出值是11维数组,argmax取得组大值的对应索引
        #索引值即分类结果,跟label比较
        train_acc+=np.sum(np.argmax(
            train_pred.cpu().data.numpy(),axis=1)==data[1].numpy())
        #计算损失函数,item返回浮点型数据
        train_loss+=batch_loss.item()
        
    #验证集 评估模型
    
    #一键搞定Batch Normalization和Dropout
    model.eval()
    #在使用pytorch时，并不是所有的操作都需要进行计算图的生成（计算过程的构建，以便梯度反向传播等操作）。
    #而对于tensor的计算操作，默认是要进行计算图的构建的，
    #在这种情况下，可以使用 with torch.no_grad():
    #强制之后的内容不进行计算图构建。
    with torch.no_grad():
        for i,data in enumerate(val_loader):
            val_pred=model(data[0].cuda())
            batch_loss=loss(val_pred,data[1].cuda())
            
            val_acc+=np.sum(np.argmax(
                val_pred.cpu().data.numpy(),axis=1)==data[1].numpy())
            val_loss+=batch_loss.detach().numpy()
            
        #打印结果
        print('[{}/{}] {:.2f} sec(s) Train Acc: {:.6f} Loss: {:.6f} | Val Acc: {:.6f} loss: {:.6f}'.format(
            epoch + 1, 
            num_epoch,
            time.time()-epoch_start_time,
            train_acc/train_set.__len__(), 
            train_loss/train_set.__len__(),
            val_acc/val_set.__len__(), 
            val_loss/val_set.__len__()))

        
    

D:\anaconda\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


RuntimeError: CUDA out of memory. Tried to allocate 128.00 MiB (GPU 0; 2.00 GiB total capacity; 1020.97 MiB already allocated; 75.35 MiB free; 1.00 GiB reserved in total by PyTorch)